### Import Libraries

In [1]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots


import folium
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns

%matplotlib inline

import math
import random
from datetime import timedelta

# color pallettes
cnf = '#393e46'
dth = '#ff2e63'
rec = '#21bf73'
act = '#fe9801'

import warnings
warnings.filterwarnings('ignore')

import plotly as py
py.offline.init_notebook_mode(connected =True)

### Read Data

In [3]:
df = pd.read_csv('D:/Data_Visualization/COVID Data/Covid-19-Preprocessed-Dataset-main/Covid-19-Preprocessed-Dataset-main/preprocessed/covid_19_data_cleaned.csv', parse_dates=['Date'])


country_daywise = pd.read_csv('D:/Data_Visualization/COVID Data/Covid-19-Preprocessed-Dataset-main/Covid-19-Preprocessed-Dataset-main/preprocessed/country_daywise.csv', parse_dates=['Date'])


countrywise = pd.read_csv('D:/Data_Visualization/COVID Data/Covid-19-Preprocessed-Dataset-main/Covid-19-Preprocessed-Dataset-main/preprocessed/countrywise.csv')


daywise = pd.read_csv('D:/Data_Visualization/COVID Data/Covid-19-Preprocessed-Dataset-main/Covid-19-Preprocessed-Dataset-main/preprocessed/daywise.csv', parse_dates=['Date'])

In [4]:
df.head()

,Date,Province/State,Country,Lat,Long,Confirmed,Recovered,Deaths,Active
0,2020-01-22,NaN,Afghanistan,33.93911,67.709953,0,0,0,0
1,2020-01-23,NaN,Afghanistan,33.93911,67.709953,0,0,0,0
2,2020-01-24,NaN,Afghanistan,33.93911,67.709953,0,0,0,0
3,2020-01-25,NaN,Afghanistan,33.93911,67.709953,0,0,0,0
4,2020-01-26,NaN,Afghanistan,33.93911,67.709953,0,0,0,0


In [ ]:
df['Province/State'] = df['Province/State'].fillna("")
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
confirmed = df[:560].groupby("Date").sum()["Confirmed"].reset_index()
recovered = df[:560].groupby("Date").sum()["Recovered"].reset_index()
deaths = df[:560].groupby("Date").sum()["Deaths"].reset_index()

In [ ]:
# recovered[500:560]

In [ ]:
df[:560].query('Country == "US"')

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = confirmed['Date'], y = confirmed['Confirmed'], mode = 'lines+markers', name = 'Confirmed', line = dict(color = 'orange', width = 0.25)))
fig.add_trace(go.Scatter(x = confirmed['Date'], y = recovered['Recovered'], mode = 'lines+markers', name = 'Recovered', line = dict(color = 'green', width = 0.25)))
fig.add_trace(go.Scatter(x = confirmed['Date'], y = deaths['Deaths'], mode = 'lines+markers', name = 'Deaths', line = dict(color = 'red', width = 0.25)))
fig.update_layout(title = 'Worldwide Covid-19 Cases', xaxis_tickfont_size = 14, yaxis = dict(title = 'Number of Cases'))

fig.show()

### Cases Density on World Map

In [ ]:
df['Date'] = df['Date'].astype(str)
df.info()

In [ ]:
df.head()

In [ ]:
fig = px.density_mapbox(df, lat = 'Lat', lon ='Long', hover_name = 'Country', hover_data = ['Confirmed','Recovered','Deaths'], animation_frame = 'Date', color_continuous_scale = 'Portland', radius = 7, zoom = 0, height = 700)
fig.update_layout(title = 'Worldwide COVID-19 Cases with Time Lapse')
fig.update_layout(mapbox_style = 'open-street-map', mapbox_center_lon = 0)

fig.show()

In [ ]:
country_daywise.head()

In [ ]:
countrywise.head()

In [ ]:
daywise.head()

### COVID on Ships : Grand Princess, Diamond Princess and MS Zaandam

In [ ]:
ship_rows = df['Province/State'].str.contains('Grand Princess') | df['Province/State'].str.contains('Diamond Princess') | df['Country'].str.contains('Grand Princess') | df['Country'].str.contains('Diamond Princess') | df['Province/State'].str.contains('MS Zaandam') | df['Country'].str.contains('MS Zaandam')
ship = df[ship_rows]

In [ ]:
df = df[~ship_rows]

In [ ]:
ship_latest = ship[ship['Date'] == max(ship['Date'])]
ship_latest

In [ ]:
ship_latest.style.background_gradient(cmap = 'Pastel1_r')

### Cases over Time with treemap

In [ ]:
temp = df[:560].groupby('Date')['Confirmed','Deaths', 'Recovered', 'Active'].sum().reset_index()
temp = temp[temp['Date']==max(temp['Date'])].reset_index(drop = True)

tm = temp.melt(id_vars = 'Date', value_vars = ['Active', 'Recovered', 'Deaths'])
fig = px.treemap(tm, path = ['variable'], values ='value', height = 300, width = 1000, color_discrete_sequence = [act,rec,dth])

fig.data[0].textinfo = 'label+text+value'
fig.show()

### Cases over time with Area Plot

In [ ]:
temp = df[:560].groupby('Date')['Active','Recovered','Deaths'].sum().reset_index()
temp = temp.melt(id_vars = 'Date', value_vars=['Active', 'Recovered', 'Deaths'], var_name = 'Case', value_name = 'Count')
# temp

fig = px.area(temp, x = 'Date', y = 'Count', color = 'Case', height = 600, title = 'Cases Over Time', color_discrete_sequence = [act,rec,dth])
fig.update_layout(xaxis_rangeslider_visible = True)
fig.show()

### Folium Maps

In [ ]:
# World-wise cases on Folium Maps

In [ ]:
temp = df[df['Date'] == '2021-08-03']
# temp

m = folium.Map(location = [0,0], tiles = 'cartodbpositron', min_zoom = 1, max_zoom = 4, zoom_start = 1)

for i in range(0,len(temp)):
    folium.Circle(location = [temp.iloc[i]['Lat'],temp.iloc[i]['Long']], color = 'crimson', fill = 'crimson',
                 tooltip = '<li><bold> Country: '+str(temp.iloc[i]['Country'])+
                 '<li><bold> Province: '+str(temp.iloc[i]['Province/State'])+
                '<li><bold> Confirmed: '+str(temp.iloc[i]['Confirmed'])+
                '<li><bold> Deaths: '+str(temp.iloc[i]['Deaths']), radius = int(temp.iloc[i]['Confirmed'])**0.5).add_to(m)
    
    
m

### Confirmed Cases with Choropleth Map

In [ ]:
fig = px.choropleth(country_daywise, locations = 'Country', locationmode = 'country names', color = np.log(country_daywise['Confirmed']), 
                   hover_name = 'Country', animation_frame = country_daywise['Date'].dt.strftime('%Y-%m-%d'), 
                   title = 'Cases over time', color_continuous_scale = px.colors.sequential.Inferno)
fig.update(layout_coloraxis_showscale = True)
fig.show()

In [ ]:
fig = px.choropleth(country_daywise, locations = 'Country', locationmode = 'country names', color = country_daywise['Confirmed'], 
                   hover_name = 'Country', animation_frame = country_daywise['Date'].dt.strftime('%Y-%m-%d'), 
                   title = 'Cases over time', color_continuous_scale = px.colors.sequential.Inferno)
fig.update(layout_coloraxis_showscale = True)
fig.show()

### Confirmed and Death Cases with Static Colormap

In [ ]:
fig_c = px.choropleth(countrywise, locations = 'Country', locationmode = 'country names', color = np.log(countrywise['Confirmed']), hover_name = 'Country', hover_data = ['Confirmed'])

temp = countrywise[countrywise['Deaths']>0]
fig_d = px.choropleth(temp, locations = 'Country', locationmode = 'country names', color = np.log(temp['Deaths']), hover_name = 'Country', hover_data = ['Deaths'])

fig = make_subplots(rows = 1, cols = 2, subplot_titles = ['Confirmed', 'Deaths'], specs = [[{'type':'choropleth'},{'type':'choropleth'}]])

fig.add_trace(fig_c['data'][0], row = 1, col = 1)
fig.add_trace(fig_d['data'][0], row = 1, col = 2)

# If you want to remove color bar
# fig.update(layout_coloraxis_showscale = False)

fig.show()

### Deaths and Recoveries per 100 Cases

In [ ]:
daywise[:560].tail()

In [ ]:
fig_c = px.bar(daywise, x = 'Date', y = 'Confirmed', color_discrete_sequence = [act])
fig_d = px.bar(daywise, x = 'Date', y = 'Deaths', color_discrete_sequence = [dth])

fig = make_subplots(rows = 1, cols = 2, shared_xaxes = False, horizontal_spacing = 0.1,
                   subplot_titles = ('Confirmed Cases','Death Cases'))

fig.add_trace(fig_c['data'][0], row = 1, col = 1)
fig.add_trace(fig_d['data'][0], row = 1, col = 2)

fig.update_layout(height = 450)
fig.show()

In [ ]:
daywise.columns

In [ ]:
fig1 = px.line(daywise, x = 'Date', y = 'Deaths / 100 Cases', color_discrete_sequence = [dth])
fig2 = px.line(daywise, x = 'Date', y = 'Recovered / 100 Cases', color_discrete_sequence = [rec])
fig3 = px.line(daywise, x = 'Date', y = 'Deaths / 100 Recovered', color_discrete_sequence = ['#8634eb'])

fig = make_subplots(rows = 1, cols = 3, shared_xaxes = False, horizontal_spacing = 0.1,
                   subplot_titles = ('Deaths / 100 Cases','Recovered / 100 Cases','Deaths / 100 Recovered'))

fig.add_trace(fig1['data'][0], row = 1, col = 1)
fig.add_trace(fig2['data'][0], row = 1, col = 2)
fig.add_trace(fig3['data'][0], row = 1, col = 3)

fig.update_layout(height= 400)
fig.show()

### New Cases and No. of Countries 

In [ ]:
fig_c = px.bar(daywise, x = 'Date', y = 'Confirmed', color_discrete_sequence = [act])
fig_d = px.bar(daywise, x = 'Date', y = 'No. of Countries', color_discrete_sequence = [dth])

fig = make_subplots(rows =1, cols = 2, shared_xaxes = False, horizontal_spacing = 0.1, subplot_titles = ("No. of New Cases per Day", "No. of Countries"))

fig.add_trace(fig_c['data'][0], row = 1, col = 1)
fig.add_trace(fig_d['data'][0], row = 1, col = 2)

fig.show()

### Top 15 Countries Case Analysis

In [ ]:
countrywise.columns

In [ ]:
top = 15

fig_c = px.bar(countrywise.sort_values('Confirmed').tail(top), x = 'Confirmed', y = 'Country', text = 'Confirmed', orientation = 'h', color_discrete_sequence = [act])

fig_d = px.bar(countrywise.sort_values('Deaths').tail(top), x = 'Deaths', y = 'Country', text = 'Deaths', orientation = 'h', color_discrete_sequence = [dth])


fig = make_subplots(rows = 5, cols =2, shared_xaxes = False, horizontal_spacing = 0.15, vertical_spacing = 0.1, subplot_titles = ('Confirmed Cases', 'Deaths Reported'))

fig.add_trace(fig_c['data'][0], col = 1, row =1)
fig.add_trace(fig_c['data'][0], col = 2, row =1)

fig.update_layout(height = 2500)
fig.show()



In [ ]:
countrywise.columns

In [ ]:
top = 15

fig_c = px.bar(countrywise.sort_values('Confirmed').tail(top), x = 'Confirmed', y = 'Country', text = 'Confirmed', orientation = 'h', color_discrete_sequence = [cnf])

fig_d = px.bar(countrywise.sort_values('Deaths').tail(top), x = 'Deaths', y = 'Country', text = 'Deaths', orientation = 'h', color_discrete_sequence = [dth])

fig_e = px.bar(countrywise.sort_values('Active').tail(top), x = 'Active', y = 'Country', text = 'Active', orientation = 'h', color_discrete_sequence = [act])

fig_f = px.bar(countrywise.sort_values('New Cases').tail(top), x = 'New Cases', y = 'Country', text = 'New Cases', orientation = 'h', color_discrete_sequence = ['#3333cc'])

fig_g = px.bar(countrywise.sort_values('1 week change').tail(top), x = '1 week change', y = 'Country', text = '1 week change', orientation = 'h', color_discrete_sequence = ['#ffff00'])

fig_h = px.bar(countrywise.sort_values('1 week % increase').tail(top), x = '1 week % increase', y = 'Country', text = '1 week % increase', orientation = 'h', color_discrete_sequence = ['#529042'])

fig_i = px.bar(countrywise.sort_values('Cases / Million People').tail(top), x = 'Cases / Million People', y = 'Country', text = 'Cases / Million People', orientation = 'h', color_discrete_sequence = ['#99ccff'])

fig_j = px.bar(countrywise.sort_values('Confirmed last week').tail(top), x = 'Confirmed last week', y = 'Country', text = 'Confirmed last week', orientation = 'h', color_discrete_sequence = ['#993399'])


fig = make_subplots(rows = 4, cols =2, shared_xaxes = False, horizontal_spacing = 0.15, vertical_spacing = 0.1, subplot_titles = ('Confirmed Cases', 'Deaths Reported','Active Cases', 'New Cases', '1 week change','1 week % increase','Cases / Million People', 'Confirmed last week'))

fig.add_trace(fig_c['data'][0], col = 1, row =1)
fig.add_trace(fig_d['data'][0], col = 2, row =1)
fig.add_trace(fig_e['data'][0], col = 1, row =2)
fig.add_trace(fig_f['data'][0], col = 2, row =2)
fig.add_trace(fig_g['data'][0], col = 1, row =3)
fig.add_trace(fig_h['data'][0], col = 2, row =3)
fig.add_trace(fig_i['data'][0], col = 1, row =4)
fig.add_trace(fig_j['data'][0], col = 2, row =4)

fig.update_layout(height = 1600)
fig.show()
